<a href="https://colab.research.google.com/github/Gehlen05/mestrado-automacao/blob/main/algoritmo-engenharia/T2_boosting_data_sintetico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor


    Implementação simplificada de uma Árvore de Decisão para regressão,
    desenvolvida para o trabalho de Boosting.

    Este modelo constrói a árvore de forma recursiva, escolhendo em cada nó
    o melhor ponto de divisão ("split") que minimiza o erro quadrático médio (MSE) das partições resultantes. O objetivo é produzir um regressor simples que sirva como base fraca para métodos de Gradient Boosting.

    Estrutura geral do método
    - A árvore é construída top-down.
    - Em cada nó, avalia-se:
        * Cada feature do dataset.
        * Todos os thresholds únicos daquela feature.
    - Para cada divisão possível:
        * Separa os dados em `left` (<= threshold) e `right` (> threshold).
        * Calcula o erro total:
              erro = MSE(left)*n_left + MSE(right)*n_right
        * Seleciona o split que minimiza esse erro.

    Critério de parada
    A recursão é interrompida quando:
        - A profundidade máxima (`max_depth`) é atingida, ou
        - O número mínimo de amostras para dividir (`min_samples_split`)
          não é satisfeito ou nenhum split produz redução de erro.

    Previsão
    A função `predict()` percorre a árvore para cada amostra de entrada,
    seguindo os thresholds até alcançar um nó terminal e retornando seu valor.

    Hiperparâmetros
    min_samples_split : int
        Número mínimo de amostras necessário para dividir um nó.
    max_depth : int
        Profundidade máxima da árvore.

    Uso no Boosting
    Este modelo foi implementado como um regressor fraco, adequado para ser combinado sequencialmente em algoritmos de Gradient Boosting,onde cada árvore aprende a partir dos resíduos da árvore anterior.



In [ ]:

class SimpleDecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=3):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.tree = None

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        best_feature, best_threshold = None, None
        best_error = float("inf")

        for feature in range(n_features):
            values = np.unique(X[:, feature])

            for threshold in values:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                error_left = self._mse(y[left_mask]) * left_mask.sum()
                error_right = self._mse(y[right_mask]) * right_mask.sum()
                error = error_left + error_right

                if error < best_error:
                    best_error = error
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _build(self, X, y, depth):
        if depth >= self.max_depth or len(y) < self.min_samples_split:
            return {"value": float(np.mean(y))}

        feature, threshold = self._best_split(X, y)

        if feature is None:
            return {"value": float(np.mean(y))}

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        return {
            "feature": int(feature),
            "threshold": float(threshold),
            "left": self._build(X[left_mask], y[left_mask], depth + 1),
            "right": self._build(X[right_mask], y[right_mask], depth + 1),
        }

    def fit(self, X, y):
        self.tree = self._build(np.asarray(X), np.asarray(y), depth=0)

    def _predict_one(self, x, node):
        if "value" in node:
            return node["value"]

        if x[node["feature"]] <= node["threshold"]:
            return self._predict_one(x, node["left"])
        else:
            return self._predict_one(x, node["right"])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in np.asarray(X)])



In [ ]:
class SimpleGradientBoostingRegressor:
    def __init__(self, n_estimators=5, learning_rate=0.1,
                 max_depth=3, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.trees = []
        self.base_value = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        # 1) Predição inicial = média de y
        self.base_value = np.mean(y)

        # predicao inicial (constante)
        y_pred = np.full_like(y, fill_value=self.base_value, dtype=float)

        # 2) Iterações de boosting
        for _ in range(self.n_estimators):
            residual = y - y_pred  # este é o erro atual

            # treinar árvore para prever o residual
            tree = SimpleDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(X, residual)

            # guardar a árvore
            self.trees.append(tree)

            # atualizar predição
            y_pred += self.learning_rate * tree.predict(X)

    def predict(self, X):
        X = np.asarray(X)
        y_pred = np.full(X.shape[0], fill_value=self.base_value, dtype=float)

        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)

        return y_pred



Os dados utilizados foram gerados de forma sintética. O modelo foi ajustado utilizando o conjunto completo de amostras e, em seguida, validado por meio das métricas R² e MSE. Essa validação permitiu comparar diretamente o desempenho do modelo implementado com o GradientBoostingRegressor da biblioteca scikit-learn.

In [ ]:

def gerar_dataset_sintetico(n=200, noise=1.0):
    X = np.random.uniform(-3, 3, size=(n, 1))
    y = 2 * (X[:, 0] ** 2) + np.random.normal(0, noise, size=n)
    return X, y


In [ ]:
X, y = gerar_dataset_sintetico()
# meu boosting
gbr = SimpleGradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.1,
    max_depth=3
)

gbr.fit(X, y)

# boosting comercial
sk_gb = GradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.1,
    max_depth=3,            # profundidade das árvores internas
    random_state=0
)
sk_gb.fit(X, y)



GradientBoostingRegressor(n_estimators=10, random_state=0)

In [ ]:
X_test, y_test = gerar_dataset_sintetico(100, 1.0)

In [ ]:
y_pred_gbr = gbr.predict(X_test)

In [ ]:
y_pred_sk_gb = sk_gb.predict(X_test)

In [ ]:
# métricas
r2_gbr = r2_score(y_test, y_pred_gbr)
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
r2_sk_gb = r2_score(y_test, y_pred_sk_gb)
mse_sk_gb = mean_squared_error(y_test, y_pred_sk_gb)



Nos experimentos, o modelo implementado mostrou desempenho muito próximo ao da implementação comercial. Em termos de erro quadrático médio (MSE), o modelo do scikit-learn obteve valor de 5.56, enquanto o modelo desenvolvido atingiu 5.69, resultando em uma diferença absoluta de apenas 0.1323. Essa proximidade indica que ambos estão aprendendo praticamente os mesmos padrões no conjunto sintético, sem divergências relevantes nas predições.
Da mesma forma, o coeficiente de determinação (R²) apresentou diferença absoluta de apenas 0.0045, reforçando que a capacidade explicativa dos dois modelos é praticamente equivalente.

In [ ]:
print(f"R²_gbr  : {r2_gbr:.4f}")
print(f"R²_gbr  : {r2_sk_gb:.4f}")
print(f"MSE_sk_gb: {mse_gbr:.4f}")
print(f"MSE_sk_gb: {mse_sk_gb:.4f}")

R²_gbr  : 0.8100
R²_gbr  : 0.8055
MSE_sk_gb: 5.5667
MSE_sk_gb: 5.6991


In [ ]:
print(f"R²_delta  : {r2_gbr-r2_sk_gb}")
print(f"MSE_delta: {mse_gbr-mse_sk_gb}")

R²_delta  : 0.004516263263799836
MSE_delta: -0.13233351280539285
